# Exercise a, b: 새로운 모델 적용

이 파일은 기존의 코드에서 새로운 모델을 적용하여 수정한 파일이다.

**Exercise a, b에 답하기 위해 사용된다.**

## 1. 패키지 Import

PyTorch 패키지를 Import 하고, 출력 형식을 설정한다.

In [11]:
import torch.optim as optim
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

## 2. 모델 정의 (수정된 부분)

기존의 모델($w \times t_u + b$)에서 새로운 모델($w_2 \times t_u^2 + w_1 \times t_u + b$)로 코드를 수정하였다.

In [12]:
# 수정: 새로운 모델 정의
def model(t_u, w1, w2, b):
    return (w2 * t_u ** 2) + (w1 * t_u) + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

## 3. 학습을 위한 함수 정의

`model` 함수에 맞게 파라미터를 최적화하는 과정을 정의한다. `model` 함수가 바뀌었음에도 `training_loop` 함수를 바뀌지 않았는데, 이는 PyTorch 라이브러리가 모델에 독립적으로 모든 Tensor에 대한 오차 역전파를 수행할 수 있도록 구현되었기 때문이다.

In [13]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
                             
        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
            
    return params

## 4. 데이터셋 준비

데이터를 랜덤 셔플한 뒤 입력 데이터에 정규화를 시행한다.

In [14]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
                    
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

## 5. 학습 진행 (수정된 부분)

기존의 `params` Tensor는 2차원이었으나, 새로운 모델은 3개의 파라미터가 필요하므로, `params`의 차원을 3차원으로 키웠다.

In [15]:
# 수정: 파라미터의 차원 수 조정
params = torch.tensor([1.0, 0.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 2000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un,
    val_t_u = val_t_un,
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 46.7604, Validation loss 231.5821
Epoch 2, Training loss 21.0734, Validation loss 84.9183
Epoch 3, Training loss 17.4891, Validation loss 15.2917
Epoch 500, Training loss 1.8162, Validation loss 12.3090
Epoch 1000, Training loss 1.8035, Validation loss 10.8250
Epoch 1500, Training loss 1.7994, Validation loss 9.8849
Epoch 2000, Training loss 1.7988, Validation loss 9.4846


tensor([ 0.0717,  0.5533, -6.0193], requires_grad=True)